In [17]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter

from sklearn.base import clone
from scipy.ndimage import convolve
from sklearn.pipeline import Pipeline
from sklearn import linear_model, metrics
from sklearn.neural_network import BernoulliRBM
from sklearn.model_selection import train_test_split

### Shift (Augment) images by 1 pixel

In [42]:
# Expanding dataset
square_size = 28
def nudge_image(X):
    """
    This produces a list of np.array 5 times bigger than the original image 
    (np array), by moving the image around by 1px to left, right, down and up
    """
    direction_vectors = [
        [[0, 1, 0],
         [0, 0, 0],
         [0, 0, 0]],

        [[0, 0, 0],
         [1, 0, 0],
         [0, 0, 0]],

        [[0, 0, 0],
         [0, 0, 1],
         [0, 0, 0]],

        [[0, 0, 0],
         [0, 0, 0],
         [0, 1, 0]]
    ]

    def shift(x, w):
        return convolve(x.reshape((square_size, square_size)), mode='constant', weights = w)

    X = ([X] + [shift(X, vector) for vector in direction_vectors])
    return X

In [43]:
curdir = os.getcwd()
dataset_dir = os.path.join(curdir, 'sudoku_dataset')
    
# extending train set
y_train = np.empty((0, 1), int)
X_train = np.empty((0, square_size * square_size), float)

for digit in range(1, 10):
    data_dir =  os.path.join(dataset_dir, 'train/' + str(digit))
    length = len(os.listdir(data_dir))
    for idx, entry in enumerate(os.scandir(data_dir)):
        
        print(f"\rdigit: {digit}, image: {idx} / {length}", end = " ")
        if (entry.path.endswith(".jpg") and entry.is_file()):
            
            img = cv2.imread(entry.path, 0)        
            extended_imgs = nudge_image(img)
            for img_mod in extended_imgs:
                y_train = np.vstack((y_train, digit))
                X_train = np.vstack((X_train, img_mod.ravel()))

digit: 9, image: 133 / 134                                                                                                                                                     

In [44]:
print(Counter(y_train[:, 0]))

Counter({8: 785, 3: 765, 1: 705, 9: 670, 4: 665, 5: 650, 7: 650, 2: 620, 6: 560})


In [45]:
dataset_dir = os.path.join(curdir, 'sudoku_dataset')

# extending test
y_test = np.empty((0, 1), int)
X_test = np.empty((0, square_size * square_size), float)

for digit in range(1, 10):
    data_dir =  os.path.join(dataset_dir, 'test/' + str(digit))
    length = len(os.listdir(data_dir))
    for idx, entry in enumerate(os.scandir(data_dir)):
        
        print(f"\rdigit: {digit}, image: {idx} / {length}", end = " ")
        if (entry.path.endswith(".jpg") and entry.is_file()):
            
            img = cv2.imread(entry.path, 0)        
            extended_imgs = nudge_image(img)
            for img_mod in extended_imgs:
                y_test = np.vstack((y_test, digit))
                X_test = np.vstack((X_test, img_mod.ravel()))
                

digit: 9, image: 44 / 45                                                                                                                            

In [46]:
print(Counter(y_test[:, 0]))

Counter({3: 260, 8: 255, 1: 235, 5: 235, 9: 225, 2: 220, 4: 215, 7: 215, 6: 190})


### Preprocessing

In [47]:
# Scaling to (0, 1] range
X_test = (X_test - np.min(X_test, 0)) / (np.max(X_test, 0) + 0.0001)
X_train = (X_train - np.min(X_train, 0)) / (np.max(X_train, 0) + 0.0001)

# random shuffling
p_test = np.random.permutation(len(X_test))
X_test, y_test = X_test[p_test], y_test[p_test]

p_train = np.random.permutation(len(X_train))
X_train, y_train = X_train[p_train], y_train[p_train]

### Training RBM + Logistic Regression

In [48]:
rbm = BernoulliRBM(random_state = 0, verbose=True)
logistic = linear_model.LogisticRegression(solver = 'newton-cg', tol = 1)

In [49]:
rbm_features_classifier = Pipeline(
    steps=[('rbm', rbm), ('logistic', logistic)])

In [50]:
rbm.n_iter = 15
logistic.C = 6000
rbm.n_components = 100
rbm.learning_rate = 0.06

In [51]:
rbm_features_classifier.fit(X_train, y_train)

[BernoulliRBM] Iteration 1, pseudo-likelihood = -86.83, time = 0.92s
[BernoulliRBM] Iteration 2, pseudo-likelihood = -70.04, time = 1.08s
[BernoulliRBM] Iteration 3, pseudo-likelihood = -65.14, time = 1.03s
[BernoulliRBM] Iteration 4, pseudo-likelihood = -64.69, time = 0.85s
[BernoulliRBM] Iteration 5, pseudo-likelihood = -60.75, time = 0.89s
[BernoulliRBM] Iteration 6, pseudo-likelihood = -68.98, time = 0.82s
[BernoulliRBM] Iteration 7, pseudo-likelihood = -61.77, time = 0.84s
[BernoulliRBM] Iteration 8, pseudo-likelihood = -53.91, time = 0.87s
[BernoulliRBM] Iteration 9, pseudo-likelihood = -56.37, time = 0.84s
[BernoulliRBM] Iteration 10, pseudo-likelihood = -53.33, time = 0.89s
[BernoulliRBM] Iteration 11, pseudo-likelihood = -54.23, time = 0.83s
[BernoulliRBM] Iteration 12, pseudo-likelihood = -56.34, time = 0.89s
[BernoulliRBM] Iteration 13, pseudo-likelihood = -51.67, time = 0.93s
[BernoulliRBM] Iteration 14, pseudo-likelihood = -52.34, time = 0.86s
[BernoulliRBM] Iteration 15, 

C:\Users\user\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Pipeline(memory=None,
         steps=[('rbm',
                 BernoulliRBM(batch_size=10, learning_rate=0.06,
                              n_components=100, n_iter=15, random_state=0,
                              verbose=True)),
                ('logistic',
                 LogisticRegression(C=6000, class_weight=None, dual=False,
                                    fit_intercept=True, intercept_scaling=1,
                                    l1_ratio=None, max_iter=100,
                                    multi_class='auto', n_jobs=None,
                                    penalty='l2', random_state=None,
                                    solver='newton-cg', tol=1, verbose=0,
                                    warm_start=False))],
         verbose=False)

### Test Classification report

In [52]:
Y_pred = rbm_features_classifier.predict(X_test)
print("Logistic regression using RBM features:\n%s\n" % (
    metrics.classification_report(y_test, Y_pred)))

Logistic regression using RBM features:
              precision    recall  f1-score   support

           1       0.97      0.99      0.98       235
           2       0.97      0.95      0.96       220
           3       0.99      1.00      1.00       260
           4       0.97      0.97      0.97       215
           5       0.95      0.98      0.96       235
           6       0.96      0.88      0.92       190
           7       1.00      0.97      0.98       215
           8       0.95      0.99      0.97       255
           9       0.97      0.98      0.97       225

    accuracy                           0.97      2050
   macro avg       0.97      0.97      0.97      2050
weighted avg       0.97      0.97      0.97      2050




### Saving model

In [53]:
import pickle
filename = 'sudoku_model.pkl'
pickle.dump(rbm_features_classifier, open(filename, 'wb'))